In [2]:
import pandas as pd
import numpy as np

In [3]:
# IMPORTAÇÃO DA BASE DE EMPRESAS ROTULADAS
empresas_rotuladas_df = pd.read_csv(r'C:\DATASETS\Empresas Rotuladas\empresas_rotuladas.csv', dtype=str, usecols=['cnpj'])
empresas_rotuladas_df

,cnpj
0,00000942000170
1,00001102000121
2,00005201000181
3,00005201000262
4,00010501000159
...,...
1559938,98593668000194
1559939,98669997000252
1559940,98670235000195
1559941,98670656000116


### Junção com a base de inidôneos e suspensos

In [4]:
# IMPORTAÇÃO DA BASE DE INIDÔNEOS E SUSPENSOS
inidoneas_e_suspensas_df = pd.read_csv(r'C:\\DATASETS\Empresas Inidoneas e Suspensas\\sancoes-inidoneas_e_suspensas.csv', dtype=str, usecols=["CPF_CNPJ"])
inidoneas_e_suspensas_df

,CPF_CNPJ
0,13459221000174
1,01276136000191
2,76258672000163
3,17015396000143
4,21545589000183
...,...
11915,53290712087
11916,15158268134
11917,15772497472
11918,89196988800


In [6]:
# Precisamos apenas dos CNPJs:
inidoneas_e_suspensas_df = inidoneas_e_suspensas_df[inidoneas_e_suspensas_df.CPF_CNPJ.str.len()==14]
inidoneas_e_suspensas_df.shape

(5150, 1)

In [8]:
# Rotula as empresas
empresas_rotuladas_df['empresa_inidoneo_suspenso'] = (empresas_rotuladas_df.merge(inidoneas_e_suspensas_df, left_on='cnpj', right_on='CPF_CNPJ', how='left', indicator=True).eval('_merge == "both"').astype(bool))
empresas_rotuladas_df.groupby(empresas_rotuladas_df.cnpj.str.len()).empresa_inidoneo_suspenso.value_counts()

cnpj  empresa_inidoneo_suspenso
14    False                        1559697
      True                             246
Name: empresa_inidoneo_suspenso, dtype: int64

Fim do processamento de Inidôneos e Suspensos.

### Junção com a base de pessoas punidas:

In [9]:
# IMPORTAÇÃO DA BASE DE PESSOAS PUNIDAS
punidas_df = pd.read_csv(r'C:\DATASETS\Empresas Punidas\punidas.csv', dtype=str, usecols=['CPF_CNPJ'])
punidas_df

,CPF_CNPJ
0,21504073000190
1,10699386000125
2,07486627000143
3,21463538000102
4,06940189000189
...,...
198,25980213805
199,30872074897
200,14403279805
201,03133444930


In [10]:
# Precisamos apenas dos CNPJs:
punidas_df = punidas_df[punidas_df.CPF_CNPJ.str.len()==14]
punidas_df.shape

(192, 1)

In [11]:
# Rotula as empresas:
empresas_rotuladas_df['empresa_punida'] = (empresas_rotuladas_df.merge(punidas_df, left_on='cnpj', right_on='CPF_CNPJ', how='left', indicator=True).eval('_merge == "both"').astype(bool))
empresas_rotuladas_df.groupby(empresas_rotuladas_df.cnpj.str.len()).empresa_punida.value_counts()

cnpj  empresa_punida
14    False             1559934
      True                    9
Name: empresa_punida, dtype: int64

Fim do processamento de pessoas punidas.

### Junção com a base de Entidades sem Fins Lucrativos Impedidas

In [12]:
# IMPORTAÇÃO DA BASE DE Entidades sem Fins Lucrativos Impedidas
esfl_impedidas_df = pd.read_csv(r'C:\DATASETS\Entidades sem Fins Lucrativos Impedidas\sem_fim_lucr_impedidas.csv', dtype=str)
esfl_impedidas_df

,CNPJ
0,06303088000105
1,01923684000166
2,03652447000133
3,03863259000154
4,07550679000131
...,...
2532,05062489000140
2533,01052752000169
2534,15243363000125
2535,07642296000193


In [13]:
# Só temos CNPJ na coluna:
print(esfl_impedidas_df.CNPJ.str.len().value_counts())

14    2537
Name: CNPJ, dtype: int64


In [15]:
# Rotula as empresas
empresas_rotuladas_df['empresa_esfl_punida'] = (empresas_rotuladas_df.merge(esfl_impedidas_df, left_on='cnpj', right_on='CNPJ', how='left', indicator=True).eval('_merge == "both"').astype(bool))
empresas_rotuladas_df.groupby(empresas_rotuladas_df.cnpj.str.len()).empresa_esfl_punida.value_counts()

cnpj  empresa_esfl_punida
14    False                  1559909
      True                        34
Name: empresa_esfl_punida, dtype: int64

Fim do processamento de entidades sem fim lucrativo punidas.

### Feature engineering:

##### Criação da variável 'empresa_sancionada', com base nas 3 variáveis anteriores.

In [17]:
empresas_rotuladas_df['empresa_sancionada'] = empresas_rotuladas_df['empresa_inidoneo_suspenso']
                                              |empresas_rotuladas_df['empresa_punida']
                                              |empresas_rotuladas_df['empresa_esfl_punida']
empresas_rotuladas_df.empresa_sancionada.value_counts()

False    1559657
True         286
Name: empresa_sancionada, dtype: int64

In [18]:
# Remover colunas não necessárias
empresas_rotuladas_df.drop(columns=['empresa_inidoneo_suspenso', 'empresa_punida', 'empresa_esfl_punida'], inplace=True)

In [19]:
# Salvando resultado:
empresas_rotuladas_df.to_csv(r'C:\DATASETS\Empresas Rotuladas\empresas_rotuladas-sancionadas.csv', index = False)